In [2]:
# Transform crop data
# Annual figures - production volume, total sales, acres planted
# North Carolina only

import pandas as pd

In [3]:
# read in crop data for NC - 22 million rows from 1866 to 2024, some data is weekly and some data is at the county level

folder = 'C:/Users/szums/usdaData/'

df = pd.read_csv(folder + 'qs.crops_20240815.txt',delimiter='\t') 
# ignore the warning about dtypes - those columns will be removed anyway

print(f'{len(df)} rows read.')

C:\Users\szums\AppData\Local\Temp\ipykernel_15800\3884519978.py:5: DtypeWarning: Columns (19,20,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder + 'qs.crops_20240815.txt',delimiter='\t')


22397301 rows read.


In [4]:
# Pare the data down to just the rows we need (state level, annual totals, production)

df1 = df[(df['AGG_LEVEL_DESC'] == 'STATE') & (df['STATE_ALPHA'] == 'NC') & (df['SOURCE_DESC'] == 'SURVEY') & (df['REFERENCE_PERIOD_DESC'] == 'YEAR')][['SHORT_DESC','YEAR','VALUE']]

print(f'pared down rows and columns to just {len(df1)} rows.')

df1.info()


pared down rows and columns to just 22723 rows.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 22723 entries, 210 to 22397290
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SHORT_DESC  22723 non-null  object
 1   YEAR        22723 non-null  int64 
 2   VALUE       22723 non-null  object
dtypes: int64(1), object(2)
memory usage: 710.1+ KB


In [5]:

# convert the VALUE column to numeric

df1['VALUE'] = df1['VALUE'].apply(lambda v: None if v[0] == "(" else v)

df1['VALUE'] = df1['VALUE'].str.replace(',','')

df1['VALUE'] = pd.to_numeric(df1['VALUE'])

df1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 22723 entries, 210 to 22397290
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SHORT_DESC  22723 non-null  object 
 1   YEAR        22723 non-null  int64  
 2   VALUE       19924 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 710.1+ KB


In [9]:
# drop the rows that have value zero

df2 = df1[df1['VALUE'] > 0]

df2.info()

# now we are down to just 19,000 rows (from 22 million)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19006 entries, 210 to 22397290
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SHORT_DESC  19006 non-null  object 
 1   YEAR        19006 non-null  int64  
 2   VALUE       19006 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 593.9+ KB


In [6]:
# next we are going to pull out values we need (appearing on different rows) and reshape them to a wide format. 
# one row per year, and lots of columns for the datapoints we need.  

# Utility functions defined here

def add_column(df1,short_desc,value_name):

    retval = df1[df1['SHORT_DESC'] == short_desc][['YEAR','VALUE']].reset_index(drop=True)
    #display(retval.head())
    
    retval.rename(columns={'VALUE':value_name},inplace=True)
    #display(retval.head())

    return  retval


def append_column(aggregatingdf,newcol):
    aggregatingdf = pd.merge(aggregatingdf,newcol,on='YEAR',how='outer')
    return aggregatingdf

In [10]:
# get a list of all the short_desc / fieldnames from which to choose

df_shortdesc = df2[['SHORT_DESC']].drop_duplicates()
df_shortdesc.to_excel(folder+'short_desc_values.xlsx',index=False)

# use this list to find the commodities to focus on

In [97]:
# lets chop off all the unneeded columns

df1 = df[(df['AGG_LEVEL_DESC'] == 'STATE') & (df['STATE_ALPHA'] == 'NC') & (df['SOURCE_DESC'] == 'SURVEY') & (df['REFERENCE_PERIOD_DESC'] == 'YEAR')][['SHORT_DESC','YEAR','VALUE']]

print('pared down df to just NC years with',len(df1),'rows.')

display(df1.head(10))

# convert value to numeric

df1['VALUE'] = df1['VALUE'].apply(lambda v: None if v[0] == "(" else v)

df1['VALUE'] = df1['VALUE'].str.replace(',','')

df1['VALUE'] = pd.to_numeric(df1['VALUE'])

display(df1.head(10))


#dfNew2['VALUE'] = pd.to_numeric(dfNew2['VALUE'])

# now let's do some reshaping.  just a few columns at first


pared down df to just NC years with 22723 rows.


,SHORT_DESC,YEAR,VALUE
210,"TOBACCO, FLUE-CURED NC BORD & SC BELT (TYPE 13...",1940,"58,000"
1309,SWEET POTATOES - ACRES HARVESTED,2012,"62,000"
3032,RYE - ACRES HARVESTED,1892,"47,000"
3034,"TOBACCO, FLUE-CURED EAST NC BELT (TYPE 12) - P...",1925,"215,369,000"
3571,"TOBACCO, FLUE-CURED NC BORD & SC BELT (TYPE 13...",1982,"1,434,271"
4639,"RYE - YIELD, MEASURED IN BU / ACRE",1992,24
7361,"COTTON - PRODUCTION, MEASURED IN 480 LB BALES",1954,"363,000"
9361,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",1983,20
11350,"HAY, (EXCL ALFALFA) - YIELD, MEASURED IN TONS ...",1935,0.86
12209,"SWEET POTATOES - PRODUCTION, MEASURED IN $",1997,"53,568,000"


,SHORT_DESC,YEAR,VALUE
210,"TOBACCO, FLUE-CURED NC BORD & SC BELT (TYPE 13...",1940,5.800000e+04
1309,SWEET POTATOES - ACRES HARVESTED,2012,6.200000e+04
3032,RYE - ACRES HARVESTED,1892,4.700000e+04
3034,"TOBACCO, FLUE-CURED EAST NC BELT (TYPE 12) - P...",1925,2.153690e+08
3571,"TOBACCO, FLUE-CURED NC BORD & SC BELT (TYPE 13...",1982,1.434271e+06
4639,"RYE - YIELD, MEASURED IN BU / ACRE",1992,2.400000e+01
7361,"COTTON - PRODUCTION, MEASURED IN 480 LB BALES",1954,3.630000e+05
9361,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",1983,2.000000e+01
11350,"HAY, (EXCL ALFALFA) - YIELD, MEASURED IN TONS ...",1935,8.600000e-01
12209,"SWEET POTATOES - PRODUCTION, MEASURED IN $",1997,5.356800e+07


In [21]:
#initialize the summary table

print('most recent year in file is',max(df1['YEAR']))
      
new_df = df1[['YEAR']].drop_duplicates()  #.reset_index(drop=True)

new_df = new_df.sort_values(by=['YEAR'])

new_df.reset_index()

print(new_df)


most recent year in file is 2024
          YEAR
713821    1866
1700686   1867
878650    1868
451472    1869
578931    1870
...        ...
51509     2020
416865    2021
2376737   2022
20304134  2023
22172604  2024

[159 rows x 1 columns]


In [22]:
# Now reshape the data so that data points are columns. We'll stick to just the same 159 rows (years)


new_df = append_column(new_df,add_column(df1,'BARLEY - ACRES PLANTED','BARLEY_ACRES'))
new_df = append_column(new_df,add_column(df1,'BARLEY - PRODUCTION, MEASURED IN $','BARLEY_$'))
new_df = append_column(new_df,add_column(df1,'BARLEY - PRODUCTION, MEASURED IN BU','BARLEY_BU  '))
new_df = append_column(new_df,add_column(df1,'BARLEY - YIELD, MEASURED IN BU / ACRE','BARLEY_BU_ACRE'))

new_df = append_column(new_df,add_column(df1,'BLUEBERRIES, TAME - ACRES HARVESTED','_ACRE'))
new_df = append_column(new_df,add_column(df1,'BLUEBERRIES, TAME, UTILIZED - PRODUCTION, MEASURED IN $','BLUEBERRIES  _$'))
new_df = append_column(new_df,add_column(df1,'BLUEBERRIES, TAME, UTILIZED - PRODUCTION, MEASURED IN LB','BLUEBERRIES  _LB'))
new_df = append_column(new_df,add_column(df1,'BLUEBERRIES, TAME, UTILIZED - YIELD, MEASURED IN LB / ACRE','BLUEBERRIES  _LB_ACRE'))

new_df = append_column(new_df,add_column(df1,'CORN - ACRES PLANTED','CORN_ACRES'))
new_df = append_column(new_df,add_column(df1,'CORN, GRAIN - PRODUCTION, MEASURED IN $','CORN_$'))
new_df = append_column(new_df,add_column(df1,'CORN, GRAIN - PRODUCTION, MEASURED IN BU','CORN_BU  '))
new_df = append_column(new_df,add_column(df1,'CORN, GRAIN - YIELD, MEASURED IN BU / ACRE','CORN_BU_ACRE'))
new_df = append_column(new_df,add_column(df1,'COTTON - ACRES PLANTED','COTTON_ACRES'))
new_df = append_column(new_df,add_column(df1,'COTTON - PRODUCTION, MEASURED IN $','COTTON_$'))
new_df = append_column(new_df,add_column(df1,'COTTON - PRODUCTION, MEASURED IN 480 LB BALES','COTTON_LB'))
new_df = append_column(new_df,add_column(df1,'COTTON - YIELD, MEASURED IN LB / ACRE','COTTON_LB_ACRE'))

new_df = append_column(new_df,add_column(df1,'HAY - ACRES HARVESTED','HAY_ACRES'))
new_df = append_column(new_df,add_column(df1,'HAY - PRODUCTION, MEASURED IN $','HAY_$'))
new_df = append_column(new_df,add_column(df1,'HAY - PRODUCTION, MEASURED IN TONS','HAY_T   '))
new_df = append_column(new_df,add_column(df1,'HAY - YIELD, MEASURED IN TONS / ACRE','HAY_T_ACRE'))

new_df = append_column(new_df,add_column(df1,'OATS - ACRES PLANTED','OATS_ACRES'))
new_df = append_column(new_df,add_column(df1,'OATS - PRODUCTION, MEASURED IN $','OATS_$'))
new_df = append_column(new_df,add_column(df1,'OATS - PRODUCTION, MEASURED IN BU','OATS_BU  '))
new_df = append_column(new_df,add_column(df1,'OATS - YIELD, MEASURED IN BU / ACRE','OATS_BU_ACRE'))

new_df = append_column(new_df,add_column(df1,'PEANUTS - ACRES PLANTED','PEANUTS_ACRES'))
new_df = append_column(new_df,add_column(df1,'PEANUTS - PRODUCTION, MEASURED IN $','PEANUTS_$'))
new_df = append_column(new_df,add_column(df1,'PEANUTS - PRODUCTION, MEASURED IN LB','PEANUTS_LB'))
new_df = append_column(new_df,add_column(df1,'PEANUTS - YIELD, MEASURED IN LB / ACRE','PEANUTS_LB_ACRE'))
new_df = append_column(new_df,add_column(df1,'PEPPERS, BELL - ACRES PLANTED','PEPPERS, BELL_ACRES'))
new_df = append_column(new_df,add_column(df1,'PEPPERS, BELL - PRODUCTION, MEASURED IN $','PEPPERS, BELL_$'))
new_df = append_column(new_df,add_column(df1,'PEPPERS, BELL - PRODUCTION, MEASURED IN CWT','PEPPERS, BELL_CWT'))
new_df = append_column(new_df,add_column(df1,'PEPPERS, BELL - YIELD, MEASURED IN CWT / ACRE','PEPPERS, BELL_CWT_ACRE'))


new_df = append_column(new_df,add_column(df1,'PUMPKINS - ACRES PLANTED','PUMPKINS_ACRES'))
new_df = append_column(new_df,add_column(df1,'PUMPKINS - PRODUCTION, MEASURED IN $','PUMPKINS_$'))
new_df = append_column(new_df,add_column(df1,'PUMPKINS - PRODUCTION, MEASURED IN CWT','PUMPKINS_CWT'))
new_df = append_column(new_df,add_column(df1,'PUMPKINS - YIELD, MEASURED IN CWT / ACRE','PUMPKINS_CWT_ACRE'))

new_df = append_column(new_df,add_column(df1,'SOYBEANS - ACRES PLANTED','SOYBEANS_ACRES'))
new_df = append_column(new_df,add_column(df1,'SOYBEANS - PRODUCTION, MEASURED IN $','SOYBEANS_$'))
new_df = append_column(new_df,add_column(df1,'SOYBEANS - PRODUCTION, MEASURED IN BU','SOYBEANS_BU  '))
new_df = append_column(new_df,add_column(df1,'SOYBEANS - YIELD, MEASURED IN BU / ACRE','SOYBEANS_BU_ACRE'))
new_df = append_column(new_df,add_column(df1,'SQUASH - ACRES PLANTED','SQUASH_ACRES'))
new_df = append_column(new_df,add_column(df1,'SQUASH - PRODUCTION, MEASURED IN $','SQUASH_$'))
new_df = append_column(new_df,add_column(df1,'SQUASH - PRODUCTION, MEASURED IN CWT','SQUASH_CWT'))
new_df = append_column(new_df,add_column(df1,'SQUASH - YIELD, MEASURED IN CWT / ACRE','SQUASH_CWT_ACRE'))


new_df = append_column(new_df,add_column(df1,'SWEET POTATOES - ACRES PLANTED','SWEET_ACRES'))
new_df = append_column(new_df,add_column(df1,'SWEET POTATOES - PRODUCTION, MEASURED IN $','SWEET_$'))
new_df = append_column(new_df,add_column(df1,'SWEET POTATOES - PRODUCTION, MEASURED IN CWT','SWEET_CWT'))
new_df = append_column(new_df,add_column(df1,'SWEET POTATOES - YIELD, MEASURED IN CWT / ACRE','SWEET_CWT_ACRE'))
new_df = append_column(new_df,add_column(df1,'TOBACCO - ACRES HARVESTED','TOBACCO_ACRES'))
new_df = append_column(new_df,add_column(df1,'TOBACCO - PRODUCTION, MEASURED IN $','TOBACCO_$'))
new_df = append_column(new_df,add_column(df1,'TOBACCO - PRODUCTION, MEASURED IN LB','TOBACCO_LB'))
new_df = append_column(new_df,add_column(df1,'TOBACCO - YIELD, MEASURED IN LB / ACRE','TOBACCO_LB_ACRE'))

new_df = append_column(new_df,add_column(df1,'WHEAT - ACRES PLANTED','WHEAT_ACRES'))
new_df = append_column(new_df,add_column(df1,'WHEAT - PRODUCTION, MEASURED IN $','WHEAT_$'))
new_df = append_column(new_df,add_column(df1,'WHEAT - PRODUCTION, MEASURED IN BU','WHEAT_BU  '))
new_df = append_column(new_df,add_column(df1,'WHEAT - YIELD, MEASURED IN BU / ACRE','WHEAT_BU_ACRE'))




In [23]:
display(new_df.shape)
display(new_df['YEAR'].value_counts())
new_df.head()

# we now have a table with one row per year, and a lot of columns!

(159, 57)

1866    1
1975    1
1968    1
1969    1
1970    1
       ..
1920    1
1921    1
1922    1
1923    1
2024    1
Name: YEAR, Length: 159, dtype: int64

,YEAR,BARLEY_ACRES,BARLEY_$,BARLEY_BU,BARLEY_BU_ACRE,_ACRE,BLUEBERRIES _$,BLUEBERRIES _LB,BLUEBERRIES _LB_ACRE,CORN_ACRES,...,SWEET_CWT,SWEET_CWT_ACRE,TOBACCO_ACRES,TOBACCO_$,TOBACCO_LB,TOBACCO_LB_ACRE,WHEAT_ACRES,WHEAT_$,WHEAT_BU,WHEAT_BU_ACRE
0,1866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,16000.0,1568000.0,7840000.0,490.0,NaN,NaN,1800000.0,6.0
1,1867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,19000.0,1550000.0,9120000.0,480.0,NaN,NaN,2240000.0,7.0
2,1868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2016000.0,52.0,20000.0,1536000.0,9200000.0,460.0,NaN,NaN,2250000.0,6.0
3,1869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1691000.0,41.0,26000.0,1509000.0,11180000.0,430.0,NaN,NaN,2925000.0,7.5
4,1870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2435000.0,59.0,32000.0,2346000.0,16640000.0,520.0,NaN,NaN,2870000.0,7.0


In [24]:
# verify that all columns have at least some data

new_df.isna().sum()


YEAR                        0
BARLEY_ACRES               62
BARLEY_$                   86
BARLEY_BU                  61
BARLEY_BU_ACRE             61
_ACRE                     142
BLUEBERRIES  _$           142
BLUEBERRIES  _LB          142
BLUEBERRIES  _LB_ACRE     148
CORN_ACRES                 60
CORN_$                     83
CORN_BU                     0
CORN_BU_ACRE                0
COTTON_ACRES               43
COTTON_$                  145
COTTON_LB                   0
COTTON_LB_ACRE              0
HAY_ACRES                  43
HAY_$                      84
HAY_T                      44
HAY_T_ACRE                 44
OATS_ACRES                 63
OATS_$                     84
OATS_BU                     0
OATS_BU_ACRE                0
PEANUTS_ACRES              43
PEANUTS_$                 117
PEANUTS_LB                 43
PEANUTS_LB_ACRE            43
PEPPERS, BELL_ACRES       133
PEPPERS, BELL_$           133
PEPPERS, BELL_CWT         133
PEPPERS, BELL_CWT_ACRE    133
PUMPKINS_A

In [26]:
# Now let's save the transformed data

new_df.to_csv(folder+'cropsANNUALNCwide.csv',index=False)